In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import my_lib

... OK (Got response in 2.25 seconds)
Writing emoji data to /root/.demoji/codes.json ...
... OK


In [2]:
nn_arch = [{"units": 256, "activation": "relu"},
           {"units": 128, "activation": "relu"},
           {"units": 2, "activation": "softmax"}]

model_input_dict = {"adaboost": {}, "elastic": {}, "rf": {}, "svm": {}, "xgboost": {"num_class": 2},
                    "nn": {"architecture": nn_arch, "num_class": 2}}

test_model_input_dict = {"adaboost": {}, "elastic": {}, "rf": {"param_grid":{"n_estimators": [10], 'max_depth': [2]}},
                         "svm": {}, "xgboost": {"num_class": 2},"nn": {"architecture": nn_arch, "num_class": 2}}

is_test = False

In [4]:
pipeline = my_lib.Pipeline()
raw_df = pipeline.load_data(is_test=is_test)
raw_df = pipeline.encode_to_category(raw_df)
raw_df = pipeline.tokenize(raw_df)
raw_df.head()

KeyboardInterrupt: 

In [8]:
len(raw_df)

4173

In [11]:
df = pipeline.train_embedding_words(raw_df)
X_train, X_test, y_train, y_test = pipeline.get_train_test(df, "embedding_vector", flatten=True, output_col="is_irrelevant")

In [12]:
if is_test is True:
    model_dict, score_dict = pipeline.train_model(test_model_input_dict, X_train, y_train, X_test, y_test)
else:
    model_dict, score_dict = pipeline.train_model(model_input_dict, X_train, y_train, X_test, y_test)


model
train model: adaboost
train score: 0.66926
test score: 0.66467

train model: elastic
train score: 0.69173
test score: 0.66826

train model: rf
train score: 0.67585
test score: 0.66108

train model: svm
train score: 0.71929
test score: 0.66826

train model: xgboost
[14:59:00] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softmax' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
train score: 0.67466
test score: 0.66587

train model: nn
Train on 3338 samples
3338/3338 [==============================] - ETA: 10s - loss: 0.6890 - accuracy: 0.7500 - ETA: 5s - loss: 0.6874 - accuracy: 0.6250  - ETA: 3s - loss: 0.6753 - accuracy: 0.6510

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:2325: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


In [13]:
for model_name, model in model_dict.items():
    model.save(f"irrevelant_{model_name}_train_{score_dict[model_name][0]:.4f}__score_{score_dict[model_name][1]:.4f}")





INFO:tensorflow:Assets written to: /app/models/irrevelant_nn_train_0.6702__score_0.6695/assets
